In [2]:
import cv2

def main():
    # Open a connection to the front camera (camera index 0)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video stream from the camera.")
        return

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame from the camera.")
            break

        # Display the resulting frame
        cv2.imshow('Video Stream', frame)

        # Press 'q' to quit the video stream
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture when everything is done
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Error: Could not read frame from the camera.


In [4]:
import cv2
import time
import os

def main():
    # Create a directory to save the photos
    if not os.path.exists('photos'):
        os.makedirs('photos')

    # Load the Haar cascade file for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Open a connection to the front camera (camera index 0)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open video stream from the camera.")
        return

    last_captured_time = time.time()

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        if not ret:
            print("Error: Could not read frame from the camera.")
            break

        # Convert the frame to grayscale (face detection works better on grayscale images)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Draw rectangles around detected faces and add a label
        for (x, y, w, h) in faces:
            # Draw rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

            # Set label below the face
            label = "Face"
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            label_y = y + h + label_size[1] + 10

            # Draw a filled rectangle behind the label text
            cv2.rectangle(frame, (x, label_y - label_size[1] - 10), (x + label_size[0], label_y + base_line - 10), (255, 0, 0), cv2.FILLED)

            # Put label text below the face
            cv2.putText(frame, label, (x, label_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

            # Take a photo every 3 seconds if a face is detected
            current_time = time.time()
            if current_time - last_captured_time >= 3:
                # Save the photo with a timestamp
                photo_filename = f"photos/face_{int(current_time)}.jpg"
                cv2.imwrite(photo_filename, frame)
                print(f"Photo saved: {photo_filename}")
                last_captured_time = current_time

        # Display the resulting frame
        cv2.imshow('Video Stream - Face Detection', frame)

        # Press 'q' to quit the video stream
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture when everything is done
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Photo saved: photos/face_1717387258.jpg
Photo saved: photos/face_1717387262.jpg
Photo saved: photos/face_1717387265.jpg
Photo saved: photos/face_1717387268.jpg
Photo saved: photos/face_1717387271.jpg
Photo saved: photos/face_1717387274.jpg
Photo saved: photos/face_1717387277.jpg


KeyboardInterrupt: 

In [13]:
import cv2
import os
import numpy as np

# Function to preprocess the images
def preprocess_images(image_paths):
    images = []
    labels = []

    for path in image_paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        images.append(image)
        labels.append(int(os.path.basename(os.path.dirname(path))))

    return images, np.array(labels)

# Function to train the face recognition model
def train_model(data_dir):
    # Create a list of all image paths
    image_paths = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_paths.append(os.path.join(root, file))

    # Preprocess the images
    images, labels = preprocess_images(image_paths)

    # Create LBPH face recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Train the recognizer
    recognizer.train(images, labels)

    # Save the trained model
    recognizer.save("trained_model.yml")

    print("Model trained and saved successfully.")

# Directory containing the dataset
data_dir = "./photos"

# Train the model
train_model(data_dir)

Model trained and saved successfully.


In [4]:
import cv2
import os
import numpy as np

# Function to preprocess the images
def preprocess_images(image_paths):
    images = []
    labels = []

    for path in image_paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        images.append(image)
        labels.append(int(os.path.basename(os.path.dirname(path))))

    return images, np.array(labels)

# Function to train the face recognition model
def train_model(data_dir):
    # Create a list of all image paths
    image_paths = []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                image_paths.append(os.path.join(root, file))

    # Preprocess the images
    images, labels = preprocess_images(image_paths)

    # Create LBPH face recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    # Train the recognizer
    recognizer.train(images, labels)

    return recognizer

# Load data containing information about each person's face
# Example: { 'id1': 'John', 'id2': 'Jane', ... }
data = {1: 'Uttam'}

# Train the face recognition model
recognizer = train_model("photos")

# Initialize some variables
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
last_captured_time = 0

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame from the camera.")
        break

    # Convert the frame to grayscale (face detection works better on grayscale images)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces and add a label
    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Take a region of interest (ROI) for face recognition
        face_roi = gray[y:y+h, x:x+w]

        # Perform face recognition
        label_id, confidence = recognizer.predict(face_roi)

        # Map label ID to name
        name = data.get(label_id, "Unknown")

        # Display name below the face
        cv2.putText(frame, name, (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Take a photo every 3 seconds if a face is detected
        current_time = cv2.getTickCount() / cv2.getTickFrequency()
        if current_time - last_captured_time >= 3:
            # Save the photo with a timestamp
            photo_filename = f"photos_temp/face_{int(current_time)}.jpg"
            cv2.imwrite(photo_filename, frame)
            print(f"Photo saved: {photo_filename}")

            last_captured_time = current_time

    # Display the resulting frame
    cv2.imshow('Video Stream - Face Detection', frame)

    # Press 'q' to quit the video stream
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture when everything is done
cap.release()
cv2.destroyAllWindows()


Photo saved: photos_temp/face_1067457.jpg
Photo saved: photos_temp/face_1067460.jpg
Photo saved: photos_temp/face_1067463.jpg
Photo saved: photos_temp/face_1067466.jpg
Photo saved: photos_temp/face_1067469.jpg
Photo saved: photos_temp/face_1067475.jpg
Photo saved: photos_temp/face_1067478.jpg
Photo saved: photos_temp/face_1067482.jpg
Photo saved: photos_temp/face_1067485.jpg
Photo saved: photos_temp/face_1067488.jpg
Photo saved: photos_temp/face_1067491.jpg
Photo saved: photos_temp/face_1067494.jpg
Photo saved: photos_temp/face_1067497.jpg
Photo saved: photos_temp/face_1067500.jpg
Photo saved: photos_temp/face_1067503.jpg
Photo saved: photos_temp/face_1067506.jpg


KeyboardInterrupt: 